In [26]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px
import geopandas as gpd
import shapely 
import numpy as np
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import dash
from dash_bootstrap_templates import load_figure_template
import plotly.graph_objects as go
import dash_daq as daq
# import sys
# from pathlib import Path
# sys.path.append(str(Path(__file__).parent))
from figure_functions import barchart_heating_systems, boxplot_co2, boxplot_investment
from constants import INJECTION_PRICE, GAS_PRICE, ELECTRICITY_PRICE, ELEC_CO2_INTENSITY, ELEC_CO2_INTENSITY_ELECTRIFICATION, COLS
from tab_layout import make_layout_scenario_maker, make_sliders
from server import app
from functions import map_epc
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from luminalysis_functions import example_cases, dynamic_co2, fit_co2

In [8]:
# df = pd.read_feather('data/archetypes_luminus_trimmed.ftr')
# def hs(scenario_name, gas, elec, oil, heating_consumption):
#     if scenario_name == 'current situation':
#         return 'Oil' if oil >= heating_consumption else 'Gasboiler'
#     elif 'hp' in scenario_name.lower() or 'warmtepomp' in scenario_name.lower():
#         return 'Heat Pump'
#     elif 'gasboiler' in scenario_name.lower():
#         return 'Gasboiler'
#     else:
#         return 'Gasboiler'

# def dhw_system(scenario_name, gas, elec, heating_system):
#     if 'heat pump boiler' in scenario_name.lower():
#         return 'HP Boiler'
#     elif heating_system == 'Heat Pump':
#         if gas == 0:
#             return 'HP Boiler'
#         else:
#             return 'Gasboiler'
#     else:
#         return 'Gasboiler'
# df['heating_system'] = df.apply(lambda x: hs(x.scenario_name,
#                                              x.gas_consumption,
#                                              x.electricity_consumption,
#                                              x.oil_consumption,
#                                              x.heating_consumption), axis = 1)
# df['dhw_system'] = df.apply(lambda x: dhw_system(x.scenario_name,
#                                              x.gas_consumption,
#                                              x.electricity_consumption,
#                                              x.heating_system), axis = 1)

In [27]:


#df = pd.read_csv('data/archetypes_luminus.csv', usecols = COLS)
df = pd.read_feather('data/archetypes_luminus_trimmed.ftr')
df['tco'] = df.total_investment_cost + df.electricity_consumption*15*ELECTRICITY_PRICE['BE'] + df.gas_consumption*GAS_PRICE['BE'] * 15+ df.total_oil_cost -df.electricity_injected * INJECTION_PRICE['BE'] * 15
df['total_co2'] = df.electricity_consumption * 0.171 + df.gas_consumption * 0.056 * 3.6 + df.oil_consumption * 0.074 * 3.6 - df.electricity_injected * 0.171
# df['mapped_epc'] = df.apply(lambda x: map_epc( epc = x.epc_ind, 
#                                                 co2 = x.total_co2 / x.total_floor_area), axis = 1)
optimal_unmodded_temp = df[df.epc_ind<=100].sort_values(by = 'tco', ascending = True).drop_duplicates(subset = 'name', keep = 'first')
optimal_unmodded_temp['dataset'] = 'unmodded'
optimal_modded_temp = df[df.epc_ind<=100].sort_values(by = 'tco', ascending = True).drop_duplicates(subset = 'name', keep = 'first')
optimal_modded_temp['dataset'] = 'modded'

In [10]:
df.total_co2

0         2712.4074
1         2425.8114
2         2180.2554
3         2180.2554
4         2425.8114
            ...    
676355    -126.1980
676356    -505.4760
676357    -505.4760
676358    -126.1980
676359    -505.4760
Name: total_co2, Length: 676360, dtype: float64

In [33]:
x_fit, y_fit_central, y_fit_electrification, fig = fit_co2(central_scenario = ELEC_CO2_INTENSITY, electrification_scenario = ELEC_CO2_INTENSITY_ELECTRIFICATION, returnplot = True)
# Plot the data and the fit

In [34]:
fig.show()

In [30]:
y_fit_central

array([0.17813384, 0.16992409, 0.1620927 , 0.15462225, 0.14749609,
       0.14069836, 0.13421392, 0.12802833, 0.12212782, 0.11649925,
       0.11113008, 0.10600837, 0.10112271, 0.09646221, 0.0920165 ,
       0.08777569, 0.08373032, 0.0798714 , 0.07619032, 0.0726789 ,
       0.06932931])

In [12]:
# # Convert dictionary to arrays
# years = np.array(list(ELEC_CO2_INTENSITY.keys()))
# central_scenario = np.array(list(ELEC_CO2_INTENSITY.values()))
# electrification_scenario = np.array(list(ELEC_CO2_INTENSITY_ELECTRIFICATION.values()))

# # Exponential decay function
# def exp_decay(x, a, b):
#     return a * np.exp(b * x)

# # Fit the central scenario data
# popt_central, pcov_central = curve_fit(exp_decay, years - 2020, central_scenario, p0=(0.171, -0.1))
# a_central, b_central = popt_central

# # Fit the electrification scenario data
# popt_electrification, pcov_electrification = curve_fit(exp_decay, years - 2020, electrification_scenario, p0=(0.171, -0.1))
# a_electrification, b_electrification = popt_electrification

# # Generate data for plotting
# x_fit = np.linspace(2020, 2040, 21)
# y_fit_central = exp_decay(x_fit - 2020, *popt_central)
# y_fit_electrification = exp_decay(x_fit - 2020, *popt_electrification)
# Plot the data and the fit
# fig = plt.figure(figsize=(10, 6))
# plt.plot(years, central_scenario, 'o', label='Central Scenario Data')
# plt.plot(x_fit, y_fit_central, '-', label=f'Central Scenario Fit: a={a_central:.3f}, b={b_central:.3f}')
# plt.plot(years, electrification_scenario, 'o', label='Electrification Scenario Data')
# plt.plot(x_fit, y_fit_electrification, '-', label=f'Electrification Scenario Fit: a={a_electrification:.3f}, b={b_electrification:.3f}')
# plt.xlabel('Year')
# plt.ylabel('CO2 Intensity (kg/kWh)')
# plt.title('CO2 Intensity Projections until 2040')
# plt.legend()
# plt.grid(True)
# plt.show()


In [13]:
df = dynamic_co2(df, x_fit, y_fit_central, start_year = 2025)

2025 0.14069835834222524
2026 0.13421391714875425
2027 0.12802832789703306
2028 0.12212781723629437
2029 0.1164992465956229
2030 0.11113008292851366
2031 0.1060083708057416
2032 0.10112270579440223
2033 0.09646220906384673
2034 0.092016503161967
2035 0.08777568890789235
2036 0.08373032334964464
2037 0.07987139873767113
2038 0.07619032246743532
2039 0.07267889794640446
2040 0.06932930634283063


In [14]:
df[['electricity_consumption', 'gas_consumption', 'total_co2', 'pv_production', 'oil_consumption', 'heating_consumption', 'domestic_hot_water_consumption',  'total_emissions']]

,electricity_consumption,gas_consumption,total_co2,pv_production,oil_consumption,heating_consumption,domestic_hot_water_consumption,total_emissions
0,1763,11959,2712.4074,0,0,8803,3155,41427.278969
1,1222,11959,2425.8114,1676,0,8803,3155,38715.706262
2,1000,11959,2180.2554,3112,0,8803,3155,36392.425590
3,1270,11959,2180.2554,3112,0,8803,3155,36392.425590
4,1329,11959,2425.8114,1676,0,8803,3155,38715.706262
...,...,...,...,...,...,...,...,...
676355,3289,0,-126.1980,4787,0,1491,1275,-1193.998006
676356,3808,0,-505.4760,7899,0,1491,1275,-4782.463557
676357,4189,0,-505.4760,7899,0,1491,1275,-4782.463557
676358,2896,0,-126.1980,4787,0,1491,1275,-1193.998006


In [15]:
testcase = df[df.epc_ind <= 100].sort_values(by = 'tco', ascending = True).drop_duplicates(subset = 'name', keep = 'first')
testcase = testcase[testcase.name == 'type_0']

In [16]:
testcase['total_co2'].values * np.array([1 for i in range(16)]).cumsum()

array([  780.885,  1561.77 ,  2342.655,  3123.54 ,  3904.425,  4685.31 ,
        5466.195,  6247.08 ,  7027.965,  7808.85 ,  8589.735,  9370.62 ,
       10151.505, 10932.39 , 11713.275, 12494.16 ])

In [17]:
testcase = df[(df.epc_ind<100) & (df.name == 'type_0')].sort_values(by = 'tco', ascending = False).drop_duplicates(subset = ['heating_system', 'dhw_system'], keep = 'first')

In [18]:
testcase[['electricity_consumption', 'gas_consumption', 'total_co2', 'pv_production', 'oil_consumption', 'heating_consumption', 'domestic_hot_water_consumption',  'total_emissions', 'heating_system', 'dhw_system', 'scenario_name']]

,electricity_consumption,gas_consumption,total_co2,pv_production,oil_consumption,heating_consumption,domestic_hot_water_consumption,total_emissions,heating_system,dhw_system,scenario_name
3162,2763,0,78.6600,3112,0,533,1275,744.226399,Heat Pump,HP Boiler,"Triple glass, incl. frame + Internal wall insu..."
2245,1488,3155,496.6830,3112,0,533,3155,8858.192966,Heat Pump,Gasboiler,"Triple glass, incl. frame + Internal wall insu..."
3153,2298,2918,616.8258,2872,0,2918,1275,9682.487341,Gasboiler,HP Boiler,"Triple glass, incl. frame + Internal wall insu..."
2161,1000,6074,993.8394,3112,0,2918,3155,17409.769590,Gasboiler,Gasboiler,"Triple glass, incl. frame + Internal wall insu..."


In [19]:
# split_scenarios = df['scenario_name'].str.split('+')
# all_entries = list(split_scenarios.values.flatten())
#set([item.strip() for sublist in all_entries for item in sublist])

In [20]:
df[(df.heating_system == 'Heat Pump') & (df.gas_consumption > 0)][['electricity_consumption', 'gas_consumption', 'total_co2', 'pv_production', 'oil_consumption', 'heating_consumption', 'domestic_hot_water_consumption',  'total_emissions', 'heating_system', 'scenario_name']]

,electricity_consumption,gas_consumption,total_co2,pv_production,oil_consumption,heating_consumption,domestic_hot_water_consumption,total_emissions,heating_system,scenario_name
17,3284,3155,1197.612,0,0,1520,3155,15489.897338,Heat Pump,Air-water HP - no floor heating
18,3206,3155,1184.274,0,0,1442,3155,15363.702426,Heat Pump,Air-water HP - floor heating
49,3203,3155,1183.761,0,0,1440,3155,15358.848776,Heat Pump,"Double glass, incl. frame + Air-water HP - no ..."
57,3036,3155,1155.204,0,0,1272,3155,15088.662235,Heat Pump,"Double glass, incl. frame + Ground-water warmt..."
59,3129,3155,1171.107,0,0,1366,3155,15239.125399,Heat Pump,"Double glass, incl. frame + Air-water HP - flo..."
...,...,...,...,...,...,...,...,...,...,...
675257,2532,3155,-87.624,7899,0,1491,3155,3329.885126,Heat Pump,"Triple glass, incl. frame + Internal wall insu..."
675258,1322,3155,368.604,3591,0,1491,3155,7646.398242,Heat Pump,"Triple glass, incl. frame + Internal wall insu..."
675259,2913,3155,-87.624,7899,0,1491,3155,3329.885126,Heat Pump,"Triple glass, incl. frame + Internal wall insu..."
675260,996,3155,368.604,3591,0,1491,3155,7646.398242,Heat Pump,"Triple glass, incl. frame + Internal wall insu..."


In [25]:
testcase[['electricity_consumption', 'electricity_injected', 'emissions_2025', 'emissions_2026', 'emissions_2027','gas_consumption', 'total_co2', 'pv_production', 'oil_consumption', 'heating_consumption', 'domestic_hot_water_consumption',  'total_emissions', 'heating_system', 'dhw_system', 'scenario_name']]

,electricity_consumption,electricity_injected,emissions_2025,emissions_2026,emissions_2027,gas_consumption,total_co2,pv_production,oil_consumption,heating_consumption,domestic_hot_water_consumption,total_emissions,heating_system,dhw_system,scenario_name
3162,2763,2303,64.721245,61.738402,58.893031,0,78.6600,3112,0,533,1275,744.226399,Heat Pump,HP Boiler,"Triple glass, incl. frame + Internal wall insu..."
2245,1488,2303,521.378838,526.663658,531.704913,3155,496.6830,3112,0,533,3155,8858.192966,Heat Pump,Gasboiler,"Triple glass, incl. frame + Internal wall insu..."
3153,2298,2131,611.765426,610.682524,609.649531,2918,616.8258,2872,0,2918,1275,9682.487341,Gasboiler,HP Boiler,"Triple glass, incl. frame + Internal wall insu..."
2161,1000,2349,1034.716315,1043.463826,1051.808186,6074,993.8394,3112,0,2918,3155,17409.769590,Gasboiler,Gasboiler,"Triple glass, incl. frame + Internal wall insu..."


In [22]:
testcase[testcase.heating_system == 'Gasboiler']

,Unnamed: 0,epc_ind,epc_label,total_co2,scenario_name,gas_consumption,oil_consumption,total_floor_area,heating_consumption,electricity_injected,...,emissions_2032,emissions_2033,emissions_2034,emissions_2035,emissions_2036,emissions_2037,emissions_2038,emissions_2039,emissions_2040,total_emissions
3153,0,80,2,616.8258,"Triple glass, incl. frame + Internal wall insu...",2918,0,97,2918,2131,...,605.156292,604.377989,603.635556,602.927340,602.251764,601.607324,600.992584,600.406176,599.846794,9682.487341
2161,0,72,2,993.8394,"Triple glass, incl. frame + Internal wall insu...",6074,0,97,2918,2349,...,1088.103870,1094.390880,1100.388137,1106.108996,1111.566194,1116.771883,1121.737655,1126.474567,1130.993166,17409.769590


In [23]:
for x in zip(testcase.heating_system.unique(), testcase.dhw_system.unique()):
    print(x)

('Heat Pump', 'HP Boiler')
('Gasboiler', 'Gasboiler')


In [24]:
new_emissions_cumulative.shape[1]

NameError: name 'new_emissions_cumulative' is not defined

In [ ]:
tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]].cumsum(axis=1).values[0]

array([1034.7163146 , 1043.46382577, 1051.80818567, 1059.76797455,
       1067.36091634, 1074.60391813, 1081.51310778, 1088.10386988,
       1094.39087997, 1100.38813723, 1106.10899566, 1111.5661938 ,
       1116.7718831 , 1121.73765499, 1126.47456667, 1130.99316574])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=years,
    y=tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]].cumsum(axis=1).values[0],
    mode='lines+markers',
    name=f'{heating_system} + {dhw_system}'
))

In [ ]:
fig = go.Figure()
for heating_system in testcase.heating_system.unique():
    for dhw_system in testcase.dhw_system.unique():
        tmp = testcase[(testcase.heating_system == heating_system) & (testcase.dhw_system == dhw_system)]
        years = np.arange(2025, 2025 + len(y_fit_central[5:]))
        total_co2_cumulative = tmp['total_co2'].values * np.array([1 for i in range(16)]).cumsum()
        new_emissions_cumulative = tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]].cumsum(axis=1)
        #new_emissions_cumulative = tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]]
        # Create the Plotly scatter plot


        # Add total_co2 cumulative emissions
        # fig.add_trace(go.Scatter(
        #     x=years,
        #     y=total_co2_cumulative,
        #     mode='lines+markers',
        #     name='Total CO2 (Original)'
        # ))
        fig.add_trace(go.Scatter(
            x=years,
            y=new_emissions_cumulative.values[0],
            mode='lines+markers',
            name=f'{heating_system} + {dhw_system}'
        ))
        fig.add_trace(go.Scatter(
            x=years,
            y=total_co2_cumulative,
            mode='lines+markers',
            name=f'standard {heating_system} + {dhw_system}'
        ))
        
# # Add newly calculated cumulative emissions
# tmp = testcase[testcase.heating_system == 'Heat Pump']
# years = np.arange(2025, 2025 + len(y_fit_central[5:]))
# #total_co2_cumulative = tmp['total_co2'].values * np.array([1 for i in range(16)]).cumsum()
# new_emissions_cumulative = tmp[[f'emissions_{year + 2025}' for year in range(len(y_fit_central[5:]))]].cumsum(axis=1)
# for i in range(new_emissions_cumulative.shape[0]):
#     fig.add_trace(go.Scatter(
#         x=years,
#         y=new_emissions_cumulative.iloc[i],
#         mode='lines+markers',
#         name=f'{tmp.heating_system.values}'
#     ))

# Update layout
fig.update_layout(
    title='Yearly Cumulative Emissions',
    xaxis_title='Year',
    yaxis_title='Cumulative Emissions (kg CO2)',
    legend_title='Emission Categories'
)

# Show the plot
fig.show()

In [ ]:
df['self_cons'] = df.pv_production